In [ ]:
import os
import secrets
from PIL import Image
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField # BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError # Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta
import re

app = Flask(__name__)
db = SQLAlchemy(app) #举例说明 instantiate the database, passing the app
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'#a secret key from flask login is needed
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'#configure 使成型/配置

login_manager = LoginManager()
login_manager.init_app(app) #initialize it:passing the flap cap to it
login_manager.login_view = "login"

@login_manager.user_loader #你要先取得user_id，才能进行CUAD
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    notes = db.relationship('Note', backref='writer', lazy='dynamic')
    
    def __repr__(self):
        return '<Task %r>' % self.id


class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
    def __repr__(self):
        return '<Task %r>' % self.id


class RegisterForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")


class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    note_body = StringField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Note Body"})
    submit = SubmitField("Add Note")


    
################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

@app.route("/")
def home():
    sqlalchemyObj = db.engine.execute('select * from product_recommend ORDER BY name')
    product_recommend = []
    for i in sqlalchemyObj:
        product_recommend.append(i)
    dataset = []
    dict={}
    for i in product_recommend:
        dict['id'] = i[0]
        dict['name'] = i[1]
        dict['url'] = i[2]
        dict['img_url'] = i[3]
        dict['desc'] = i[4]
        dict['old_price'] = i[5]
        dict['new_price'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('index.html', dataset=dataset)

@app.route("/guide_all")
def guide_all():
    return render_template('guide_all.html')

@app.route("/guide_ingre")
def guide_ingre():

    sqlalchemyObj = db.engine.execute('select * from guide WHERE type=1 ORDER BY name')
    guide_ingre = []
    for i in sqlalchemyObj:
        guide_ingre.append(i)
 
    dataset = []
    dict={}
    for i in guide_ingre:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_ingre.html', dataset=dataset)

@app.route("/guide_steps")
def guide_steps():
    
    return render_template('guide_steps.html')

@app.route("/guide_classify")
def guide_classify():
    
    return render_template('guide_classify.html')

@app.route("/guide_blogger")
def guide_blogger():

    sqlalchemyObj = db.engine.execute('select * from blogger ORDER BY name')
    guide_blogger = []
    for i in sqlalchemyObj:
        guide_blogger.append(i)
    
    dataset = []
    dict={}
    for i in guide_blogger:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_blogger.html', dataset=dataset)

@app.route("/guide_blogger_oil")
def guide_blogger_oil():

    sqlalchemyObj = db.engine.execute('select * from blogger WHERE type=1 ORDER BY name')
    guide_blogger_oil = []
    for i in sqlalchemyObj:
        guide_blogger_oil.append(i)
    
    dataset = []
    dict={}
    for i in guide_blogger_oil:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_blogger_oil.html', dataset=dataset)

@app.route("/guide_blogger_dry")
def guide_blogger_dry():

    sqlalchemyObj = db.engine.execute('select * from blogger WHERE type=2 ORDER BY name')
    guide_blogger_dry = []
    for i in sqlalchemyObj:
        guide_blogger_dry.append(i)
    
    dataset = []
    dict={}
    for i in guide_blogger_dry:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_blogger_dry.html', dataset=dataset)

@app.route("/guide_blogger_mixed")
def guide_blogger_mixed():

    sqlalchemyObj = db.engine.execute('select * from blogger WHERE type=3 ORDER BY name')
    guide_blogger_mixed = []
    for i in sqlalchemyObj:
        guide_blogger_mixed.append(i)
    
    dataset = []
    dict={}
    for i in guide_blogger_mixed:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_blogger_mixed.html', dataset=dataset)

@app.route("/guide_blogger_neutral")
def guide_blogger_neutral():

    sqlalchemyObj = db.engine.execute('select * from blogger WHERE type=4 ORDER BY name')
    guide_blogger_neutral = []
    for i in sqlalchemyObj:
        guide_blogger_neutral.append(i)
    
    dataset = []
    dict={}
    for i in guide_blogger_neutral:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('guide_blogger_neutral.html', dataset=dataset)


@app.route("/problem_dou")
def problem_dou():

    sqlalchemyObj = db.engine.execute('select * from problem WHERE type=1 ORDER BY name')
    problem_dou = []
    for i in sqlalchemyObj:
        problem_dou.append(i)
    
    dataset = []
    dict={}
    for i in problem_dou:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('problem_dou.html', dataset=dataset)

@app.route("/problem_hei")
def problem_hei():

    sqlalchemyObj = db.engine.execute('select * from problem WHERE type=2 ORDER BY name')
    problem_hei = []
    for i in sqlalchemyObj:
        problem_hei.append(i)
  
    dataset = []
    dict={}
    for i in problem_hei:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('problem_hei.html', dataset=dataset)

@app.route("/problem_quan")
def problem_quan():

    sqlalchemyObj = db.engine.execute('select * from problem WHERE type=3 ORDER BY name')
    problem_quan = []
    for i in sqlalchemyObj:
        problem_quan.append(i)
  
    dataset = []
    dict={}
    for i in problem_quan:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['author'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('problem_quan.html', dataset=dataset)

@app.route("/product_oil")
def product_oil():
    
    sqlalchemyObj = db.engine.execute('select * from product_oil ORDER BY name')
    product_oil = []
    for i in sqlalchemyObj:
        product_oil.append(i)
 
    dataset = []
    dict={}
    for i in product_oil:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_oil.html', dataset=dataset)

@app.route("/product_oil_1")
def product_oil_1():
    
    sqlalchemyObj = db.engine.execute('select * from product_oil WHERE step=1 ORDER BY name')
    product_oil_1 = []
    for i in sqlalchemyObj:
        product_oil_1.append(i)

    dataset = []
    dict={}
    for i in product_oil_1:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_oil_1.html', dataset=dataset)

@app.route("/product_oil_2")
def product_oil_2():
    
    sqlalchemyObj = db.engine.execute('select * from product_oil WHERE step=2 ORDER BY name')
    product_oil_2 = []
    for i in sqlalchemyObj:
        product_oil_2.append(i)

    dataset = []
    dict={}
    for i in product_oil_2:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_oil_2.html', dataset=dataset)

@app.route("/product_oil_3")
def product_oil_3():
    
    sqlalchemyObj = db.engine.execute('select * from product_oil WHERE step=3 ORDER BY name')
    product_oil_3 = []
    for i in sqlalchemyObj:
        product_oil_3.append(i)

    dataset = []
    dict={}
    for i in product_oil_3:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_oil_3.html', dataset=dataset)

@app.route("/product_dry")
def product_dry():
    
    sqlalchemyObj = db.engine.execute('select * from product_dry ORDER BY name')
    product_dry = []
    for i in sqlalchemyObj:
        product_dry.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_dry:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_dry.html', dataset=dataset)

@app.route("/product_dry_1")
def product_dry_1():
    
    sqlalchemyObj = db.engine.execute('select * from product_dry WHERE step=1 ORDER BY name')
    product_dry_1 = []
    for i in sqlalchemyObj:
        product_dry_1.append(i)
    dataset = []
    dict={}
    for i in product_dry_1:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_dry_1.html', dataset=dataset)

@app.route("/product_dry_2")
def product_dry_2():
    
    sqlalchemyObj = db.engine.execute('select * from product_dry WHERE step=2 ORDER BY name')
    product_dry_2 = []
    for i in sqlalchemyObj:
        product_dry_2.append(i)
    dataset = []
    dict={}
    for i in product_dry_2:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_dry_2.html', dataset=dataset)

@app.route("/product_dry_3")
def product_dry_3():
    
    sqlalchemyObj = db.engine.execute('select * from product_dry WHERE step=3 ORDER BY name')
    product_dry_3 = []
    for i in sqlalchemyObj:
        product_dry_3.append(i)
    dataset = []
    dict={}
    for i in product_dry_3:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_dry_3.html', dataset=dataset)

@app.route("/product_mixed")
def product_mixed():

    sqlalchemyObj = db.engine.execute('select * from product_mixed ORDER BY name')
    product_mixed = []
    for i in sqlalchemyObj:
        product_mixed.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_mixed:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_mixed.html', dataset=dataset)

@app.route("/product_mixed_1")
def product_mixed_1():

    sqlalchemyObj = db.engine.execute('select * from product_mixed WHERE step=1 ORDER BY name')
    product_mixed_1 = []
    for i in sqlalchemyObj:
        product_mixed_1.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_mixed_1:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_mixed_1.html', dataset=dataset)

@app.route("/product_mixed_2")
def product_mixed_2():

    sqlalchemyObj = db.engine.execute('select * from product_mixed WHERE step=2 ORDER BY name')
    product_mixed_2 = []
    for i in sqlalchemyObj:
        product_mixed_2.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_mixed_2:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_mixed_2.html', dataset=dataset)

@app.route("/product_mixed_3")
def product_mixed_3():

    sqlalchemyObj = db.engine.execute('select * from product_mixed WHERE step=3 ORDER BY name')
    product_mixed_3 = []
    for i in sqlalchemyObj:
        product_mixed_3.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_mixed_3:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_mixed_3.html', dataset=dataset)

@app.route("/product_neutral")
def product_neutral():
    
    sqlalchemyObj = db.engine.execute('select * from product_neutral ORDER BY name')
    product_neutral = []
    for i in sqlalchemyObj:
        product_neutral.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_neutral:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_neutral.html', dataset=dataset)

@app.route("/product_neutral_1")
def product_neutral_1():
    
    sqlalchemyObj = db.engine.execute('select * from product_neutral WHERE step=1 ORDER BY name')
    product_neutral_1 = []
    for i in sqlalchemyObj:
        product_neutral_1.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_neutral_1:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_neutral_1.html', dataset=dataset)

@app.route("/product_neutral_2")
def product_neutral_2():
    
    sqlalchemyObj = db.engine.execute('select * from product_neutral WHERE step=2 ORDER BY name')
    product_neutral_2 = []
    for i in sqlalchemyObj:
        product_neutral_2.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_neutral_2:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_neutral_2.html', dataset=dataset)

@app.route("/product_neutral_3")
def product_neutral_3():
    
    sqlalchemyObj = db.engine.execute('select * from product_neutral WHERE step=3 ORDER BY name')
    product_neutral_3 = []
    for i in sqlalchemyObj:
        product_neutral_3.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in product_neutral_3:
        dict['id'] = i[0]
        dict['step'] = i[1]
        dict['name'] = i[2]
        dict['url'] = i[3]
        dict['img_url'] = i[4]
        dict['desc'] = i[5]
        dict['old_price'] = i[6]
        dict['new_price'] = i[7]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('product_neutral_3.html', dataset=dataset)

@app.route("/community")
def community():
    sqlalchemyObj = db.engine.execute('select * from community ORDER BY name')
    community = []
    for i in sqlalchemyObj:
        community.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in community:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]       
        dict['level'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('community.html', dataset=dataset)

@app.route("/community_cao")
def community_cao():
    sqlalchemyObj = db.engine.execute('select * from community WHERE type=1 ORDER BY name')
    community_cao = []
    for i in sqlalchemyObj:
        community_cao.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in community_cao:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]       
        dict['level'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('community_cao.html', dataset=dataset)  

@app.route("/community_lei")
def community_lei():
    sqlalchemyObj = db.engine.execute('select * from community WHERE type=2 ORDER BY name')
    community_lei = []
    for i in sqlalchemyObj:
        community_lei.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in community_lei:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]       
        dict['level'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('community_lei.html', dataset=dataset)  

@app.route("/community_improve")
def community_improve():
    sqlalchemyObj = db.engine.execute('select * from community WHERE type=3 ORDER BY name')
    community_improve = []
    for i in sqlalchemyObj:
        community_improve.append(i)
    # print(product_oil)
    dataset = []
    dict={}
    for i in community_improve:
        dict['id'] = i[0]
        dict['type'] = i[1]
        dict['name'] = i[2]       
        dict['level'] = i[3]
        dict['url'] = i[4]
        dict['img_url'] = i[5]
        dict['desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())
    print(dataset)
    
    return render_template('community_improve.html', dataset=dataset)  

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("view_notes"))
    
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route('/new-notes', methods=['GET','POST'])
@login_required
def new_note():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_note = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('view_notes'))
    return render_template('new_notes.html', title='New Note', form=form)


@app.route('/my-notes', methods=['GET','POST'])
@login_required
def view_notes():
    notes = Note.query.filter_by(writer=current_user).all()
    return render_template('my_notes.html', notes=notes, title='My Notes')


@app.route('/delete-note/<int:note_id>', methods=['GET',"POST"])
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('view_notes'))



################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################

# APPLICATION TEST RUN AT PORT 9003

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 9088)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/kwantszmei/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9088/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 22:47:25] "GET / HTTP/1.1" 200 -


[{'id': 5, 'name': '【Sephora Collection】 Coach x Sephora Collection Tea Rose Eye Mask Set (Limited Edition)', 'url': 'https://www.sephora.hk/products/sephora-collection-coach-x-sephora-collection-tea-rose-eye-mask-set-limited-edition', 'img_url': 'https://image-optimizer-hk.production.sephora-asia.net/images/product_images/closeup_1_Product_3378872165149-Sephora-Collection-Coach-x-Sephora-Collection-Tea-Rose-Eye-Mask-Se_ff4101d7eeb53eb5bbd6d545159a98444cf13e25_1614248433.png', 'desc': '改善暗啞', 'old_price': 'HK$130.00 ', 'new_price': 'HK$110.50 '}, {'id': 1, 'name': '【The Ordinary】Hyaluronic Acid 2% + B5', 'url': 'https://deciem.com/en-hk/theordinary/mas-rdn-hyaluronic-acid-2pct-b5.html', 'img_url': 'https://deciem.com/dw/image/v2/BFKJ_PRD/on/demandware.static/-/Sites-deciem-master/default/dwa736cdde/Images/products/The%20Ordinary/rdn-hyaluronic-acid-2pct-b5-30ml.png', 'desc': '面霜前使用，保湿更有效', 'old_price': '56.00 HKD', 'new_price': '43.12 HKD'}, {'id': 4, 'name': '【多芬】潤澤水嫩潔面泡泡', 'url': 'ht

127.0.0.1 - - [13/Dec/2021 22:47:27] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 22:47:27] "GET /static/assets/img/about.jpg HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 22:47:28] "GET /register HTTP/1.1" 200 -
